# HEATHOTSAUCE.COM SCRAPE

https://heathotsauce.com

In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [ ]:
# scrape all 19 pages to get names and urls

names = []
urls = []

# go to each page (print progress)
for i in range(1, 19):
    url = 'https://heathotsauce.com/collections/all?page=' + str(i)
    page = requests.get(url)
    print (i, page)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #in each page, scrape the names
    for j in soup.findAll('span', class_='title product-name-custom'):
        names.append(j.get_text())
    for k in soup.findAll(itemprop = 'url'):
        urls.append(k.get('href'))

In [ ]:
#combine names and urls into a df
names_urls = pd.DataFrame({'names':names, 'urls':urls})

# add url to start of urls
names_urls['urls'] = names_urls['urls'].apply(lambda x: 'https://heathotsauce.com' + str(x))

In [ ]:
names_urls

In [ ]:
## Go to each unique url, grab name, price, manufacturer, description

## NOTE: I requested too many timesand was timed out. 
## Do this in batches, save as different df, then concat together

name = []
mfc = []
price = []
desc = []
urls = []

for n, url in enumerate(names_urls['urls'][:21]):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
#     name
    try:
        name.append(soup.find('h1', class_='product_name').get_text())
    except:
        name.append('')
#     manufacturer
    try:
        mfc.append(soup.find('p', class_='vendor').get_text())
    except:
        mfc.append('')
#     price
    try:
        price.append(soup.find('span', class_='money').get_text())
    except:
        price.append('')
#     description
    try:
        desc.append(soup.find('div', class_='description').get_text())
    except:
        desc.append('')
#     url
    try:
        urls.append(url)
    except:
        urls.append('')
    
#     print progress, if response != 200, stop kernel. you have been timed out
    print (n, page)

df_1 = pd.DataFrame({'names':name,
                   'mfc':mfc,
                   'price':price,
                   'desc':desc,
                   'url':urls})

In [ ]:
df_1

In [ ]:
# combine all df batches. strip manufacturers of \n
full_df = df_1
full_df['mfc'] = full_df['mfc'].apply(lambda x: x.strip())
full_df['desc'] = full_df['desc'].apply(lambda x: x.strip())

# remove gift sets and empty rows
full_df = full_df[full_df['mfc'] != 'Gift Set']
full_df = full_df[full_df['desc'] != '']
full_df.reset_index(inplace=True, drop=True)

# save df as csv
full_df.to_csv('hot_sauces.csv', encoding='utf-8', index=False)

In [ ]:
full_df